<a href="https://colab.research.google.com/github/Junperr/TIPE/blob/main/AI/all_station.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation

Ici on télécharge les modules pour la cartographie

In [ ]:
!pip install geopandas
!pip install pandas  #geopanda dependencies
!pip install fiona   #geopanda dependencies
!pip install shapely #geopanda dependencies
!pip install pyproj  #geopanda dependencies
!pip install rtree   #geopanda dependencies

On importe tout les modules que l'ont utilise par la suite et on connecte l'environnement au drive

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import sqlite3 as sql
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

On donne l'acces à google collab à certain fichiers du drive

In [ ]:
!ls /content/drive/MyDrive/Info/Tipe/sismic1.db
!ls /content/drive/MyDrive/Info/Tipe/Shapefiles/polbnda_jpn.shp
!ls /content/drive/MyDrive/Info/Tipe/Shapefiles/coastl_jpn.shp

# Parcours des données

Ci dessous l'on définit nos set de données la partie pour entraîner le modèle et celle pour vérifier que l'on peut généraliser notre modèle (on verifie sa précision sur des données qu'il n'a jamais vu)

In [ ]:
con=sql.connect('/content/drive/MyDrive/Info/Tipe/sismic1.db')
table_earth = pd.read_sql("""SELECT * FROM seismes order by mag desc""",con)
table_sta = pd.read_sql("""SELECT * FROM station """,con)
table_info = pd.read_sql("""SELECT * FROM infos """,con)
print('infos :',table_info.head(),'earthquake :',table_earth.head(),"station :",table_sta.head(),sep='\n'+'####'+'\n')
# station_list = pd.read_sql("""SELECT PGA,PGV,mean,std,PGA_UD,PGA_EW,PGA_NS FROM infos """,sql.connect('sismic1.db'))

dftrain = pd.read_sql("""
SELECT st.code,earthquake_id,mag,PGA,PGV,mean,std,PGA_UD,PGA_EW,PGA_NS FROM infos join station as st
on st.code = infos.station_code join seismes as se
on se.id==infos.earthquake_id
Group By earthquake_id
Order by mag DESC """,sql.connect('/content/drive/MyDrive/Info/Tipe/sismic1.db'))
print('dftrain',dftrain.head(),sep='\n')
# print(dftrain,dftrain.loc[0], sep='\n')
print(len(dftrain))

Quelque graphes pour visualiser la distributions des données:

In [ ]:
dfstat_seisme = pd.read_sql("""
SELECT AVG(nbr_record) as nbr_moy_record,count(*) as nbr_earth,mag from (
SELECT count(*) as nbr_record,se.id,se.mag FROM infos join  seismes as se
on se.id==infos.earthquake_id
group by se.id
--having nbr_record >20
)
group by mag
order by mag desc""",sql.connect('/content/drive/MyDrive/Info/Tipe/sismic1.db'))
#dftrain['nbr_record']
ax = dfstat_seisme.plot.scatter(x='mag', y='nbr_moy_record', c='nbr_earth',grid=True,use_index=True,figsize=(15,7),title='test',xlabel="magnitude",cmap='winter',colorbar=True)
ax.set_xticks([0.5*x for x in range(4,17)])
ax.set_xticklabels([0.5*x for x in range(4,17)])
plt.show()

In [ ]:
dftrain.mag.hist(bins=80).set_xlabel("magnitude")

# Data Normalisation and selection


Ici l'on a un problème contrairement au premier modèle ou l'input était de shape (3,) (mag,long,lat) et l'output de shape (1,) le PGA de la station pour lequel le modèle était entraîné.
Ici l'on veut le PGA (ou PGV cela ne change rien dans les faits) à toute les stations, cependant un séisme n'est pas toujours détecté par les mêmes stations et donc pas le même nombre de station il faut donc modifier nos données pour que pour tout couple (earthquake,station) on ait une valeur pour le PGA, pour l'entrainement du modèle comme la vérification a posteriori, on va donc la compléter par des 0 (avant de normer) quand on n'a pas de donnée.

Ci-dessous on donne un nouvel identifiant

In [ ]:
order_sta5col = pd.read_sql(""" SELECT Count(*) as nb_seismes,station_code,lat,long,network FROM infos Join station
                            On infos.station_code = station.code
                            Group By station_code
                            Order By nb_seismes DESC """,con)
order_sta=order_sta5col['station_code'].tolist()
data_set = pd.read_sql(""" SELECT id,mag,long,lat FROM seismes """,con)
#print(data_set)
#code_to_int=[]
#for x in range(len(data_set)):
#  code_to_int.append( order_sta.index(data_set['station_code'][x]))
#data_set.pop('station_code')
#data_set['station_code']=code_to_int

data_set_normed= data_set.copy()
for x in data_set.keys()[1:]:#aucun interet de normaliser l'id elle nous sert pour associer les valeurs de PGA après
  data_set_normed[x] = (data_set[x]-data_set[x].mean())/data_set[x].std()
print(data_set,data_set_normed,sep='\n')

In [ ]:
nq=len(data_set) # number of earthquake in the data base
ns=len(order_sta) # number of station in the data base
tab=np.zeros([nq,ns])
# un tableau qui pour l'indice [i,j] donne le PGA enregistrer par la station étant
# à l'indice j dans order_sta durant le seisme d'id i+1 (les indices commence à 0)
# alors que les id à 1

for i in range (0,nq):
    station_detected = pd.read_sql("""Select PGA,station_code From infos
    Where earthquake_id = {}
    """.format(i+1),con)
    # pour chaque séismes on récupère les stations qui l'ont enregistré
    for j in station_detected['station_code']:
      tab[i,order_sta.index(j)] = pd.read_sql("""Select PGA From infos
    Where earthquake_id = {} and station_code = '{}'
    """.format(i+1,j),con)['PGA'][0]
    # et on met à jour le tableau avec la valeur de PGA de sismic1
print(tab)

In [ ]:
# cette cellule permet juste de s'assurer que notre programme marche bien
count=[0 for x in range (ns)]
for i in range (nq):
  for j in range (ns):
    if tab[i,j]!=0:
      count[j]+=1
print(count)
print(order_sta5col)

In [ ]:
print(tab)
way_back=[tab.mean(),tab.std()]
tab = (tab - tab.mean())/tab.std()
print(tab,tab.mean(),tab.std(),sep='\n')

In [ ]:
data_set_normed = data_set_normed.sample(frac=1).reset_index(drop=True)
# shuffle our data_set, drop=True mean we do not keep old index on a new column
n=len(data_set_normed)
data_train = data_set_normed[:int(0.90*n)]
data_eval = data_set_normed[int(0.90*n):]
data_eval = data_eval.reset_index(drop=True)
print(data_train)
train_id = data_train.pop('id').tolist()
eval_id = data_eval.pop('id').tolist()
print(eval_id,train_id,sep='\n')
result_train=np.zeros([len(train_id),ns])
result_eval=np.zeros([len(eval_id),ns])
print(len(train_id))
for i in range(len(train_id)):
  result_train[i] = tab[train_id[i]-1]
for i in range(len(eval_id)):
  result_eval[i] = tab[eval_id[i]-1]
print(result_train,result_eval,sep='\n')

On voit que la contruction du tableau correspond bien à ce que l'on voulait.

Maintenant pour chaque seismes on a une sortie de shape (ns,) on a donc plus le problème précédent

# Model

On créer notre model:

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(2000, activation=tf.nn.elu,input_shape=[3]),
  tf.keras.layers.Dense(5000, activation=tf.nn.elu),
  tf.keras.layers.Dense(units=ns, input_shape=[ns])
])

On le compile:


In [ ]:
model.compile(loss="mae",
              optimizer=tf.keras.optimizers.Adam(0.001),metrics=['mae','mse','accuracy'],run_eagerly=True)
list_optimizers = ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
# on peut en essayer des différents (pour l'instant Adam)

In [ ]:
model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

history = model.fit(data_train, result_train, epochs=500,validation_split=0.2, verbose=False, callbacks=[early_stop])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  #plt.ylim([0, 100])
  #plt.xlim([0,50])
  plt.xlabel('Epoch')
  plt.ylabel('Error [PGA]')
  plt.legend()
  plt.grid(True)
plot_loss(history)

On cherche ici à trouver un "bon" séisme dans le set de donnée que le model n'a encore jamais vus pour ensuite comparer les carte réelle et celle predit par le model

In [ ]:
order_quake2col = pd.read_sql("""
Select Count(*) As nbr_sta,earthquake_id From infos
Group By earthquake_id
Order By nbr_sta Desc
""",con)
order_quake = order_quake2col['earthquake_id'].tolist()
print(order_quake[0],order_quake2col['earthquake_id'][0])

trick_sort = [order_quake.index(x) for x in eval_id]
print(order_quake)
print(data_eval)
data_eval['id']=trick_sort
print(data_eval)
data_eval.sort_values('id',axis=0,ascending=True,inplace=True)
print(data_eval)
sorted_id = data_eval.pop('id')
real_id_sorted = [order_quake[x] for x in sorted_id]
data_eval['id'] = real_id_sorted
print(data_eval)
print(data_eval[15:20])

In [ ]:
guineapig = data_eval[0:1]
print(guineapig,way_back)
id_guineapig = guineapig.pop('id')
true_values = tab[id_guineapig]
result_to_plot = model.predict(guineapig)
true_values = (true_values * way_back[1]) + way_back[0]
result_to_plot = (result_to_plot * way_back[1]) + way_back[0]
print(true_values,result_to_plot,result_to_plot.shape,sep='\n')

# Cartographie

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import shapely.geometry as geo
from scipy.interpolate import griddata
import numpy as np
from scipy import ndimage
import rtree
con=sql.connect('/content/drive/MyDrive/Info/Tipe/sismic1.db')

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(32,9))
X = np.linspace(0,np.pi,150)
def plot_f(f):
  plt.plot(X,f(X))
plt.subplot(1,2,1)
plot_f(np.cos)
plt.subplot(1,2,2)
plot_f(np.sin)

Les variables (globales) nécessaire à map_plot
(elle ne sont pas définie de façon locale pour éviter de les définir à nouveaux à chaque appel)

In [ ]:
japan_f = gpd.read_file("/content/drive/MyDrive/Info/Tipe/Shapefiles/polbnda_jpn.shp")
japan_c = gpd.read_file("/content/drive/MyDrive/Info/Tipe/Shapefiles/coastl_jpn.shp")
# open two different japan map shape one with area one without


# read sql database for station and create panda dataframe
crs='EPSG:8994'
# système de coordonnées


# accel_colors=['accel','#ffffff','#99b3e6','#4073d9','#33ccad','#06d039','#99ff60','#ffff00','#ff9a34','#ff6275','#990000']
accel_colors=['accel',[255,255,255],[150,180,230],[70,130,230],[50,200,180],[30,220,80],[150,255,90],[255,255,0],[255,170,70],[255,80,120],[150,0,0]]
accel_colors[1:] = [[x[0]/255,x[1]/255,x[2]/255] for x in accel_colors[1:]]
print(accel_colors)

In [ ]:
"""x = np.arange(1, 10)
y = x.reshape(-1, 1)
h = x * y

cs = plt.contourf(h, levels=[10, 30, 50],
    colors=['#808080', '#A0A0A0', '#C0C0C0'], extend='both')
cs.changed()
plt.colorbar()"""

In [ ]:
#fig,ax = plt.subplots(figsize=(40,40))
#s_pos=pd.read_sql("""SELECT long,lat,network,PGA FROM station as s join infos as i
#                  on s.code=i.station_code
#                  where i.earthquake_id=3""",con)
# read sql database for station and create panda dataframe
# système de coordonnées

"""
geometry = [geo.Point(s_pos["long"][i],s_pos["lat"][i]) for i in range (len(s_pos["long"]))]

geo_s_pos = gpd.GeoDataFrame(s_pos,crs=crs,geometry=geometry)

s_pos_s=pd.read_sql('SELECT lat,long FROM seismes where id=3',\
            con)
geometry_s = [geo.Point(s_pos_s["long"][i],s_pos_s["lat"][i]) for i in range (len(s_pos_s["long"]))]
geo_s_pos_s = gpd.GeoDataFrame(s_pos_s,crs=crs,geometry=geometry_s)

japan_f.plot(ax=ax,color="w",edgecolor="w",zorder=1) # map of Japan areas
japan_c.plot(ax=ax,color="k",edgecolor="none",linewidth=0.5,zorder=3) # physical border of Japan
geo_s_pos[geo_s_pos['network'] == 'kik'].plot(column="PGA", ax = ax, marker = '.',\
            markersize = 100 ,label = 'kik station',cmap='jet')

geo_s_pos[geo_s_pos['network'] == 'knt'].plot(column="PGA", ax = ax, marker = '^',\
        markersize = 80, label = 'knt station', cmap='jet')
geo_s_pos_s.plot(ax=ax,marker='*',color='red',markersize=90,label='sismic event')
plt.legend(loc=4,fontsize=40,markerscale=6)
ax.set_ylim([min(geo_s_pos['lat'])-1,max(geo_s_pos['lat'])+1])
ax.set_xlim([min(geo_s_pos['long'])-1,max(geo_s_pos['long'])+1])
cmap=plt.get_cmap('jet')
bounds = np.arange(min(geo_s_pos['PGA']),max(geo_s_pos['PGA']),cmap.N)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=cmap.N)
scalar = plt.cm.ScalarMappable(norm=norm,cmap=cmap)
plt.colorbar(scalar,cmap='jet')"""

In [ ]:
def map_plot (quake_id=93, predict = False, drop_null = True, fig = None, ax = None, map_back=True, cmap=None, colors_name = accel_colors, centered_map=True, interpolate=False, n=100,msize=50):
    """

    Parameters:
        map_back : Bool, optional
            Si True affiche la carte en fond sinon uniquement les points.
            The default is True.
        cmap : Colormap, optional
            To force a colormap. The default is None.
        colors_name : List, optional
            [string,arg*] arg are colors. The default is accel_colors.
        interpolate: Bool, optional
            Si True affiche le rendu avec une interpollation réaliser avec n
                points par axes

    USE:
        Rendu de l'accéleration maximal d'un séisme sur le japon
    Returns
    -------
    None.

    """

    def unormalize (tab):
        return (tab * way_back[1]) + way_back[0]

    def recup_date (text):
        day,hour = text.split(' ')
        hour = hour[:-3]
        return day,hour
    if predict :
        try :
            ind_eval = eval_id.index(quake_id)
            print("This earthake is in evaluation set")
            model_input = data_eval[ind_eval:ind_eval+1][:-1]
        except :
            ind_train = train_id.index(quake_id)
            print("This earthake is in training set")
            model_input = data_train[ind_train:ind_train+1]
        result_model = model.predict(model_input)[0]
        # return (result_model,unormalize(result_model))
        sta_pos = {'PGA': unormalize(result_model),'geometry':[],'long':[],'lat':[],'network':[]}
    else :
        sta_pos = {'PGA': unormalize(tab[quake_id-1]),'geometry':[],'long':[],'lat':[],'network':[]}
    for i in range (ns):
        sta_pos['geometry'].append(geo.Point((order_sta5col['long'][i],order_sta5col['lat'][i])))
        sta_pos['long'].append(order_sta5col['long'][i])
        sta_pos['lat'].append(order_sta5col['lat'][i])
        sta_pos['network'].append(order_sta5col['network'][i])
    print(sta_pos)
    geo_sta_pos = gpd.GeoDataFrame(sta_pos,crs=crs,geometry=sta_pos['geometry'])
    pga_null = geo_sta_pos[geo_sta_pos['PGA'] == 0]
    print('**********',pga_null,sep='\n')
    geo_sta_pos.drop(pga_null.index,inplace=True)
    geo_sta_pos.reset_index(drop=True,inplace=True)
    center_pos = pd.read_sql(f'SELECT lat,long,date,mag,depth FROM seismes where id={quake_id}',con)
    geometry_center = [geo.Point(center_pos["long"][i],center_pos["lat"][i]) for i in range (len(center_pos["long"]))]
    # en pratique i=0 je ne met pas 0 par défault si jamais je veux afficher plusieurs seismes
    # il faudrait dans ce cas modifier la requete sql
    center_geo = gpd.GeoDataFrame(center_pos,crs=crs,geometry=geometry_center)
    if fig == None and ax == None :
        fig,ax = plt.subplots(figsize=(18,18))
    ax.set_facecolor('#d3ffff')
    if centered_map :
        ax.set_ylim([min(geo_sta_pos['lat'])-1,max(geo_sta_pos['lat'])+1])
        ax.set_xlim([min(geo_sta_pos['long'])-1,max(geo_sta_pos['long'])+1])
    if map_back:
        japan_f.plot(ax=ax,color="w",edgecolor="w",zorder=1) # map of Japan areas
        japan_c.plot(ax=ax,color="k",edgecolor="none",linewidth=0.5,zorder=3) # physical border of Japan
        # affiche la carte du Japon en fond

    if cmap==None and colors_name!=None:
        try:
            create_cmap1(colors_name[1:],colors_name[0])
            cmap=colors_name[0]
        except:
            cmap=colors_name[0]

    # création de la colormap si elle n'est pas déjà créée
    cmap1=plt.get_cmap(cmap,10)
    # on récupère uniquement 10 couleurs en réalité celles de la liste accel_colors
    # il y a une meilleure facon de faire la colormap mais ce que j'ai fais est
    # suffisant
    # bounds = np.array([0.0, 0.2, 1.0, 4.0, 15.0, 50.0, 100.0, 200.0, 400.0, 750.0, 2000.0])
    bounds = np.array([0.03, 0.25, 1.0, 3.95, 15.0, 57.0, 111.5, 218.0, 425.50, 830.0, 2500.0])
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=10)
    scalar = plt.cm.ScalarMappable(norm=norm,cmap=cmap1)
    bar = plt.colorbar(scalar, fraction=0.03,ax = ax, drawedges=True, spacing='uniform', ticks=[0.2,0.5,1.0,2.0,5.0,10.0,20.0,50.0,100.0,200.0,500.0,1000.0,2000.0])
    bar.ax.tick_params(direction='inout', length=6, width=2, colors='k', grid_color='k', grid_alpha=0.5)
    bar.set_label(label='PGA (gal)',fontsize=20)
    bar.ax.set_yticklabels(['0.2','0.5','1.0','2.0','5.0','10.0','20.0','50.0','100.0','200.0','500.0','1000.0','2000.0'],fontsize=15)
# met la barre à l'échelle et créer la norme et ajoute des graduations

    geo_sta_pos[geo_sta_pos['network'] == 'kik'].plot(column="PGA", ax = ax, marker = ',',\
                    markersize = 85,edgecolor="black",linewidth=0.5,vmax=2000.0 ,norm=norm,label = 'KiK-NET station',cmap=cmap1,zorder=4)

    geo_sta_pos[geo_sta_pos['network'] == 'knt'].plot(column="PGA", ax = ax, marker = '^',\
             markersize=100 ,edgecolor="black",linewidth=0.5,norm=norm, vmax=2000.0, label = 'K-NET station', cmap=cmap1,zorder=5)
    # on affiche les sismogrpahes du réseaux kik et knt qui ont percu le seisme
    center_geo.plot(ax=ax,marker='*',color='red',markersize=200,label='Epicenter',zorder=6)
    plt.ylabel('Latitude',fontsize=20)
    yticks = ax.get_yticks()
    ax.set_yticklabels([str(x) for x in yticks], fontsize=15)
    plt.xlabel('Longitude',fontsize=20)
    xticks = ax.get_xticks()
    ax.set_xticklabels([str(x) for x in xticks], fontsize=15)
    ax.tick_params(axis='both', direction='out', length=6, width=2, colors='k', grid_color='k', grid_alpha=0.5)
    ax.legend(loc=4,fontsize=24,markerscale=2.5)
    if predict :
        ax.set_title(label = 'Predicted Peak Acceleration Contour Map', fontsize=30,loc='center')
    else:
        ax.set_title(label = 'Real Peak Acceleration Contour Map', fontsize=30,loc='center')
    day,hour = recup_date(center_pos['date'][0])
    plt.suptitle(t = f" {day} {hour} {str( center_pos['lat'][0] )}N {str(center_pos['long'][0])}E {str(center_pos['depth'][0])}km {str(center_pos['mag'][0])}M ", x = 0.5, y= 0.065, fontsize = 30)
    # ajout de l'épicentre et de la legende


    ### interpolation à partir d'ici
    if interpolate:
        ratio = (max(geo_sta_pos['lat'])-min(geo_sta_pos['lat']))/(max(geo_sta_pos['long'])-min(geo_sta_pos['long']))
        if ratio <= 1:
            xgrid,ygrid = np.mgrid[min(geo_sta_pos['long']):max(geo_sta_pos['long']):n*1j , min(geo_sta_pos['lat']):max(geo_sta_pos['lat']):ratio*n*1j]
        else:
            xgrid,ygrid = np.mgrid[min(geo_sta_pos['long']):max(geo_sta_pos['long']):n*1j/ratio , min(geo_sta_pos['lat']):max(geo_sta_pos['lat']):n*1j]
        # meshgrid prend en argument le nombre de point selon un axe et non le nombre total
        # on ajuste donc le nombre de point pour qu'ils soient répartit uniformément
        # avec comme nombre maximum de point selon un axe n
        point = np.array([(geo_sta_pos['long'][i],geo_sta_pos['lat'][i]) for i in range (len(geo_sta_pos['long']))])
        # on recupere les coordonnées de tout les sismographes pour les utiliser
        # en tant que valeurs de références pour l'interpolation
        interpolated_values = griddata(point,np.array(geo_sta_pos['PGA']),(xgrid,ygrid),method='linear',fill_value=0.0)
        # on créer une grille de n points sur le plus petit rectangle contenant tout les sismographes
        # print(len(xgrid),len(xgrid[0]),len(ygrid),len(ygrid[0]),sep='\n')
        dataframe = {'values' : [],'geometry':[]}
        for i in range (len(xgrid)):
            for j in range (len(xgrid[0])):
                dataframe['values'].append(interpolated_values[i,j])
                dataframe['geometry'].append(geo.Point((xgrid[i,j],ygrid[i,j])))
        dataframe['geometry']=gpd.GeoSeries(dataframe['geometry'])
        geo_values = gpd.GeoDataFrame(dataframe,crs=crs,geometry=dataframe['geometry'])
        geo_values1 = geo_values.overlay(japan_f,how='intersection')
        # # print(geo_values.head())
        geo_values1.plot(column="values", ax = ax, marker = '.',\
                        markersize = msize, norm=norm, vmax=2000.0, label = 'knt station', cmap=cmap1,zorder=2)
        # inprogress(ax,geo_values,map_back)

    pass

def create_cmap1(color_list,name='test'):
    """
    PARAMETER:
        color_list : list
            list of colors.
        name : str, optional
            the name of your futur colormap. The default is 'test'.
    USE:
        Create a color map from a list of color in the order given in color_list
    Return:
        None
    """
    cmap = colors.LinearSegmentedColormap.from_list(name, color_list)
    plt.register_cmap(name, cmap)


In [ ]:
# print(order_sta5col.head(),tab,sep='\n')
# fig, axs = plt.subplots(1, 2,figsize=(40,20)) 6668
map_plot(quake_id=3,predict=False,interpolate=True,centered_map=True,n=400)

In [ ]:
map_plot(quake_id=8288,predict=True,interpolate=True,centered_map=True,n=300)

In [ ]:
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(40,18))
# plt.scatter([1,2,3,4,5],[1,2,3,4,5])
map_plot(quake_id=8288,predict=False,fig=fig,ax=ax1,interpolate=False,centered_map=False,n=300)
#plt.scatter([1,2,3,4,5],[1,2,3,4,5])
map_plot(quake_id=8288,predict=True,fig=fig,ax=ax2,interpolate=False,centered_map=False,n=300)

# Stop there

In [ ]:
 # print(data_eval,result_eval,sep='\n')
X = [x for x in range (0,len(data_eval))]
# for x in X :
#   print(data_eval[x:x+1],result_eval[x:x+1])
model_predict_single = np.array([model.predict(data_eval[x:x+1]) for x in X])

In [ ]:
fig,ax = plt.subplots(figsize=(20,15))
for x in X :
  ax.plot(x,model_predict_single[x][0] ,"k*",markersize=10)
  ax.plot(x,result_eval[x:x+1],"b8",markersize=10)
plt.grid()
plt.ylabel("PGA",fontsize=40)
plt.legend(['predicted values',"real values"],fontsize=25,markerscale=4)

In [ ]:
X1 = [x for x in range (0,len(data_train))]
model_predict_single_train = np.array([model.predict(data_train[x:x+1]) for x in X1])
fig1,ax1 = plt.subplots(figsize=(20,15))
for x in X1 :
  ax1.plot(x,model_predict_single_train[x][0] ,"k*",markersize=10)
  ax1.plot(x,result_train[x:x+1],"b8",markersize=10)
plt.grid()
plt.ylabel("PGA",fontsize=40)
plt.legend(['predicted values',"real values"],fontsize=25,markerscale=4)

Idée à faire plus tard :
- travailler sur des batch d'input au lieu de les traiter 1 par 1
- Utiliser des modèle preentrainé